<center><img src="https://assets.datacamp.com/production/repositories/6005/datasets/0c64652120b80cfe2762012d7252b439138be223/piggy_bank.jpg" alt="Piggy bank" width="600" height="500"></center>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf)!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll provide the database design script as a `.sql` file that they can then run. 

You have been asked to design a database that will have three tables:

## `client`
| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `text` | Whether the client's credit is in default |
| `housing` | `text` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `text` | Whether the client has an existing personal loan |

## `campaign`
| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `integer` | Campaign ID |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `text` | Outcome of the previous campaign |
| `campaign_outcome` | `integer` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

## `economics`
| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

In [86]:
# Start coding...
import pandas as pd
import numpy as np

In [87]:
# Load the data
marketing = pd.read_csv("bank_marketing.csv")

print(marketing.head(), marketing.columns)

   client_id  age        job  marital    education credit_default housing  \
0          0   56  housemaid  married     basic.4y             no      no   
1          1   57   services  married  high.school        unknown      no   
2          2   37   services  married  high.school             no     yes   
3          3   40     admin.  married     basic.6y             no      no   
4          4   56   services  married  high.school             no      no   

  loan    contact month  ...  campaign  pdays  previous     poutcome  \
0   no  telephone   may  ...         1    999         0  nonexistent   
1   no  telephone   may  ...         1    999         0  nonexistent   
2   no  telephone   may  ...         1    999         0  nonexistent   
3   no  telephone   may  ...         1    999         0  nonexistent   
4  yes  telephone   may  ...         1    999         0  nonexistent   

   emp_var_rate cons_price_idx  cons_conf_idx  euribor3m  nr_employed   y  
0           1.1         93.9

In [88]:
# Split the data
client = marketing[["client_id", "age", "job", "marital", "education", "credit_default", "housing", "loan"]]

campaign = marketing[["client_id", "campaign", "contact", "duration", "pdays", "previous", "poutcome", "y", "day", "month"]]

economics = marketing[["client_id", "emp_var_rate", "cons_price_idx", "euribor3m", "nr_employed"]]

print(client.head(), campaign.head(), economics.head())

   client_id  age        job  marital    education credit_default housing loan
0          0   56  housemaid  married     basic.4y             no      no   no
1          1   57   services  married  high.school        unknown      no   no
2          2   37   services  married  high.school             no     yes   no
3          3   40     admin.  married     basic.6y             no      no   no
4          4   56   services  married  high.school             no      no  yes    client_id  campaign    contact  duration  pdays  previous     poutcome   y  \
0          0         1  telephone       261    999         0  nonexistent  no   
1          1         1  telephone       149    999         0  nonexistent  no   
2          2         1  telephone       226    999         0  nonexistent  no   
3          3         1  telephone       151    999         0  nonexistent  no   
4          4         1  telephone       307    999         0  nonexistent  no   

   day month  
0   13   may  
1   19   

In [89]:
# Rename, clean, create, and delete columns
# Renaming columns
client.rename(columns={"client_id": "id"}, inplace=True)

campaign.rename(columns={"duration": "contact_duration",
                         "previous": "previous_campaign_contacts",
                         "y": "campaign_outcome",
                         "campaign": "number_contacts",
                         "poutcome": "previous_outcome"},
                inplace=True)

economics.rename(columns={"euribor3m": "euribor_three_months", "nr_employed": "number_employed"}, inplace=True)

print(client.head(), campaign.head(), economics.head())

   id  age        job  marital    education credit_default housing loan
0   0   56  housemaid  married     basic.4y             no      no   no
1   1   57   services  married  high.school        unknown      no   no
2   2   37   services  married  high.school             no     yes   no
3   3   40     admin.  married     basic.6y             no      no   no
4   4   56   services  married  high.school             no      no  yes    client_id  number_contacts    contact  contact_duration  pdays  \
0          0                1  telephone               261    999   
1          1                1  telephone               149    999   
2          2                1  telephone               226    999   
3          3                1  telephone               151    999   
4          4                1  telephone               307    999   

   previous_campaign_contacts previous_outcome campaign_outcome  day month  
0                           0      nonexistent               no   13   may  

C:\Users\Happiness Nleweoha\AppData\Local\Temp\ipykernel_18168\2802937158.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client.rename(columns={"client_id": "id"}, inplace=True)
C:\Users\Happiness Nleweoha\AppData\Local\Temp\ipykernel_18168\2802937158.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign.rename(columns={"duration": "contact_duration",
C:\Users\Happiness Nleweoha\AppData\Local\Temp\ipykernel_18168\2802937158.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [90]:
# Cleaning columns
client["education"] = client["education"].str.replace(".", "_")
client["education"] = client["education"].replace("unknown", np.NaN)

# Clean job column
client["job"] = client["job"].str.replace(".", "")

# Change campaign_outcome to binary values
campaign["campaign_outcome"] = campaign["campaign_outcome"].map({"yes": 1, "no": 0})

# Convert poutcome to binary values
campaign["previous_outcome"] = campaign["previous_outcome"].replace("nonexistent", np.NaN)
campaign["previous_outcome"] = campaign["previous_outcome"].map({"success": 1, "failure": 0})

# Add campaign_id column
campaign["campaign_id"] = 1

print(client, campaign)

C:\Users\Happiness Nleweoha\AppData\Local\Temp\ipykernel_18168\117784036.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  client["education"] = client["education"].str.replace(".", "_")
C:\Users\Happiness Nleweoha\AppData\Local\Temp\ipykernel_18168\117784036.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client["education"] = client["education"].str.replace(".", "_")
C:\Users\Happiness Nleweoha\AppData\Local\Temp\ipykernel_18168\117784036.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

          id  age          job  marital            education credit_default  \
0          0   56    housemaid  married             basic_4y             no   
1          1   57     services  married          high_school        unknown   
2          2   37     services  married          high_school             no   
3          3   40        admin  married             basic_6y             no   
4          4   56     services  married          high_school             no   
...      ...  ...          ...      ...                  ...            ...   
41183  41183   73      retired  married  professional_course             no   
41184  41184   46  blue-collar  married  professional_course             no   
41185  41185   56      retired  married    university_degree             no   
41186  41186   44   technician  married  professional_course             no   
41187  41187   74      retired  married  professional_course             no   

      housing loan  
0          no   no  
1        

C:\Users\Happiness Nleweoha\AppData\Local\Temp\ipykernel_18168\117784036.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  client["job"] = client["job"].str.replace(".", "")
C:\Users\Happiness Nleweoha\AppData\Local\Temp\ipykernel_18168\117784036.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign["campaign_outcome"] = campaign["campaign_outcome"].map({"yes": 1, "no": 0})
C:\Users\Happiness Nleweoha\AppData\Local\Temp\ipykernel_18168\117784036.py:12: SettingWithCopyWarning: 
A value is trying to 

In [91]:
# Add year column
campaign["year"] = "2022"

# Capitalize month and day columns
campaign["month"] = campaign["month"].str.capitalize()

# Convert day to string
campaign["day"] = campaign["day"].astype(str)

# Add last_contact_date column
campaign["last_contact_date"] = campaign["year"] + "-" + campaign["month"] + "-" + campaign["day"]

# Convert to datetime
campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], infer_datetime_format="%Y-%b-%a")

# Drop unnecessary columns
campaign = campaign.drop(columns=["month", "year", "day"])

print(campaign)

C:\Users\Happiness Nleweoha\AppData\Local\Temp\ipykernel_18168\2297594395.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign["year"] = "2022"
C:\Users\Happiness Nleweoha\AppData\Local\Temp\ipykernel_18168\2297594395.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign["month"] = campaign["month"].str.capitalize()
C:\Users\Happiness Nleweoha\AppData\Local\Temp\ipykernel_18168\2297594395.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

       client_id  number_contacts    contact  contact_duration  pdays  \
0              0                1  telephone               261    999   
1              1                1  telephone               149    999   
2              2                1  telephone               226    999   
3              3                1  telephone               151    999   
4              4                1  telephone               307    999   
...          ...              ...        ...               ...    ...   
41183      41183                1   cellular               334    999   
41184      41184                1   cellular               383    999   
41185      41185                2   cellular               189    999   
41186      41186                1   cellular               442    999   
41187      41187                3   cellular               239    999   

       previous_campaign_contacts  previous_outcome  campaign_outcome  \
0                               0               Na

C:\Users\Happiness Nleweoha\AppData\Local\Temp\ipykernel_18168\2297594395.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], infer_datetime_format="%Y-%b-%a")


In [92]:
# Save tables to individual csv files
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)

display(client, campaign, economics)

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes
...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional_course,no,yes,no
41184,41184,46,blue-collar,married,professional_course,no,no,no
41185,41185,56,retired,married,university_degree,no,yes,no
41186,41186,44,technician,married,professional_course,no,no,no


,client_id,number_contacts,contact,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,campaign_id,last_contact_date
0,0,1,telephone,261,999,0,NaN,0,1,2022-05-13
1,1,1,telephone,149,999,0,NaN,0,1,2022-05-19
2,2,1,telephone,226,999,0,NaN,0,1,2022-05-23
3,3,1,telephone,151,999,0,NaN,0,1,2022-05-27
4,4,1,telephone,307,999,0,NaN,0,1,2022-05-03
...,...,...,...,...,...,...,...,...,...,...
41183,41183,1,cellular,334,999,0,NaN,1,1,2022-11-30
41184,41184,1,cellular,383,999,0,NaN,0,1,2022-11-06
41185,41185,2,cellular,189,999,0,NaN,0,1,2022-11-24
41186,41186,1,cellular,442,999,0,NaN,1,1,2022-11-17


,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0
...,...,...,...,...,...
41183,41183,-1.1,94.767,1.028,4963.6
41184,41184,-1.1,94.767,1.028,4963.6
41185,41185,-1.1,94.767,1.028,4963.6
41186,41186,-1.1,94.767,1.028,4963.6


In [93]:
# Store and print database_design
client_table = """
CREATE TABLE client (
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

campaign_table = """
CREATE TABLE campaign (
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

economics_table = """
CREATE TABLE economics (
    client_id SERIAL references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""

In [97]:
# Structuring the dictionary
database_design = {"client": client, "campaign": campaign, "economics": economics}

print(database_design)

{'client':           id  age          job  marital            education credit_default  \
0          0   56    housemaid  married             basic_4y             no   
1          1   57     services  married          high_school        unknown   
2          2   37     services  married          high_school             no   
3          3   40        admin  married             basic_6y             no   
4          4   56     services  married          high_school             no   
...      ...  ...          ...      ...                  ...            ...   
41183  41183   73      retired  married  professional_course             no   
41184  41184   46  blue-collar  married  professional_course             no   
41185  41185   56      retired  married    university_degree             no   
41186  41186   44   technician  married  professional_course             no   
41187  41187   74      retired  married  professional_course             no   

      housing loan  
0          no   no 